# Model 2 (complex)
The main difference between this model and the previous ones is that here we consider 2 traits: the cell size $m(t)$ and its protein content $p(t)$. We call $\underline{x}$ the vector
$$
\underline{x} = \begin{pmatrix} m\\ p\end{pmatrix} 
$$

As before, the traits evolution and the cell division are governed by $g(\underline{x})$ and $h(p)$ respectively, which are defined as 
$$
\begin{aligned}
g(\underline{x})&=\omega_1m\begin{pmatrix} 1\\ c\end{pmatrix} \\
h(p)&=
    \begin{cases}
    0   & , \, p<u \\
    \omega_2\frac{p+v}{u+v} & , \, p\geq u
    \end{cases}
\end{aligned}
$$
From $g(\underline{x})$ we can notice that the cell size still grows exponentially and the protein content also follows this evolution, scaled by the factor $c$. As $c$ doesn't have a real meaning, we set it to $1$. 

Moreover, in this model the condition under which the cell can divide is that it contains a minimum amount of a specific type of protein, which we call $u$. If $p\geq u$ the cell can divide, otherwise it can not. Unlike [model 1](./model_1.ipynb) we do not have any conditions on the size of the cell for the division to take place and $h$ depens only on $p$.

The initial conditions for $m(t)$ and $p(t)$ are
$$
\begin{aligned}
    p(t=0) &= 0 \\
    m(t=0) &= m_b
\end{aligned}
$$

The division process in this model occurs in the following way
$$
\begin{pmatrix} m \\ p\end{pmatrix} \rightarrow \begin{pmatrix} \text{f}m \\ 0\end{pmatrix} + \begin{pmatrix} (1-\text{f})m \\ 0\end{pmatrix}
$$
where for now we keep $\text{f}=0.5$.

In [ ]:
# Start writing code here...

# References
- Held J, Lorimer T, Pomati F, Stoop R, Albert C. Second-order phase transition in phytoplankton trait dynamics. _Chaos_. 2020;30(5):053109. doi:10.1063/1.5141755 
- emcee documentation: https://emcee.readthedocs.io/en/stable/

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=52dba48d-3798-4665-95fc-01a96804955b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>